In [47]:
from typing import NamedTuple, Tuple, Dict
from __future__ import annotations
from dataclasses import dataclass
import itertools


RAW = """F10
N3
F7
R90
F11"""



class Action(NamedTuple):
    instructions: str
    value: int
    
    @staticmethod
    def parse_action(row:str)-> Action:
        return Action(instructions=row[0],
                        value=int(row[1:]))
@dataclass
class XY():
    x: int
    y: int
    
            
class Ship:
    def __init__(self, inputs:str):
        self.actions = [Action.parse_action(row)
                        for row in inputs.split("\n")]
        self.facing = 'E'
        self.axis = {'E':'x','W':'x',
                     'N':'y', 'S':'y'}
        self.pos = XY(x=0, y=0)
    
    def turn_ship(self, ins=str, value=int):
        """
        turn ship for instructions "L" or "R"
        """
        clock = ['N', 'E', 'S', 'W']
        anticlock = ['N', 'W', 'S', 'E']
       
        if ins == 'L':
            lookup = anticlock
        else:
            lookup = clock

        excess = lookup.index(self.facing) + value/90 + 1
        cycle = itertools.cycle(lookup)
        while excess:
            new_fac = next(cycle)
            excess -= 1
        self.facing = new_fac

    
    def read_actions(self):
        for ins, val in self.actions:
            #print('ins is ', ins, 'val is', val, 'facing', self.facing)
            change_sign = ('W','S')
            if ins in change_sign or self.facing in change_sign and ins == 'F': #turn value sign
                val = -val
            if ins in ('R','L'):
                self.turn_ship(ins=ins, value=val)
            elif ins == 'F':
                if self.axis[self.facing] == 'x':
                    self.pos.x += val
                else:
                    self.pos.y += val
            elif ins in ('E', 'W'):
                self.pos.x += val
            elif ins in ('N', 'S'):
                self.pos.y += val
            else:
                raise ValueError('invalid instruction')
            #print(self.pos)
    @property
    def manhattan_distance(self) -> int:
        return abs(self.pos.x) + abs(self.pos.y)
                    
                
                
            

ship = Ship(inputs=RAW)
ship.read_actions()
assert ship.manhattan_distance == 25

In [48]:
with open("puzzle_inputs/day12.txt") as f:
    PUZZLE = f.read()
ship = Ship(inputs=PUZZLE)
ship.read_actions()
print(ship.manhattan_distance)

1007


## Part 2

In [51]:
from typing import NamedTuple, Tuple, Dict
from __future__ import annotations
from dataclasses import dataclass
import itertools


RAW = """F10
N3
F7
R90
F11"""

class Action(NamedTuple):
    instructions: str
    value: int
    
    @staticmethod
    def parse_action(row:str)-> Action:
        return Action(instructions=row[0],
                        value=int(row[1:]))
@dataclass
class XY():
    x: int
    y: int
    
class Ship:
    def __init__(self, inputs:str):
        self.actions = [Action.parse_action(row)
                        for row in inputs.split("\n")]
        self.axis = {'E':'x','W':'x',
                     'N':'y', 'S':'y'}
        self.pos = XY(x=0, y=0) 
        self.wp = XY(x=10, y=1) # waypoint start at 10E 1N
    
    def rotate_waypoint(self, ins:str, value:int):
        """
        rotate waypoint for ship for instructions "L" or "R"
        """
        clock = ['N', 'E', 'S', 'W']
        anticlock = ['N', 'W', 'S', 'E']
        
        if ins == 'L':
            lookup = anticlock
        else:
            lookup = clock
        
        wx, wy = self.wp.x, self.wp.y
        
        if wx > 0:
            facing_x = 'E'
        else:
            facing_x = 'W'
        if wy > 0 :
            facing_y = 'N'
        else:
            facing_y = 'S'
        
        excess = lookup.index(facing_x) + value/90 + 1
        cycle = itertools.cycle(lookup)
        
        while excess:
            new_fac_x = next(cycle)
            excess -= 1
        #print("new_fac_x", new_fac_x)
        if new_fac_x == 'E':
            new_wx = abs(wx)
        elif new_fac_x == 'W':
            new_wx = -abs(wx)
        elif new_fac_x == 'N':
            new_wy = abs(wx)
        elif new_fac_x =='S':
            new_wy = -abs(wx)

        excess = lookup.index(facing_y) + value/90 + 1
        cycle = itertools.cycle(lookup)
        
        while excess:
            new_fac_y = next(cycle)
            excess -= 1
        #print("new_fac_y", new_fac_y)
        if new_fac_y == 'E':
            new_wx = abs(wy)
        elif new_fac_y == 'W':
            new_wx = -abs(wy)
        elif new_fac_y == 'N':
            new_wy = abs(wy)
        elif new_fac_y == 'S':
            new_wy = -abs(wy)
        #print(ins, value)
        #print("before", self.wp)
        self.wp = XY(x= new_wx, y=new_wy)
        #print("after", self.wp)

    def read_actions(self):
        for ins, val in self.actions:
            #print(self.pos, self.wp)
            #print(ins, val)

            if ins in ('W', 'S'):
                val = -val
            
            if ins == 'F': 
                dx, dy  = self.wp.x, self.wp.y
                self.pos = XY(x = self.pos.x + (dx * val),
                              y = self.pos.y + (dy * val))
            elif ins in ('R','L'):
                self.rotate_waypoint(ins=ins, value=val) 
            elif ins in ('W', 'E'):
                self.wp.x += val
            elif ins in ('N', 'S'):
                self.wp.y += val
            else:
                raise ValueError(f'invalid instruction {ins}')
                
    @property
    def manhattan_distance(self) -> int:
        return abs(self.pos.x) + abs(self.pos.y)
                    
                
ship = Ship(inputs=RAW)
ship.read_actions()
assert ship.manhattan_distance == 286

In [52]:
with open("puzzle_inputs/day12.txt") as f:
    PUZZLE = f.read()
ship = Ship(inputs=PUZZLE)
ship.read_actions()
print(ship.manhattan_distance)

41212
